In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

/Users/pradyumna/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# 3.1. Hierarchical GLAM estimation and out of sample prediction

## Load data

In [4]:
# Load data
sufix = '_individual_Dislike_NoBin_Gamma-11_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Dislike_NoBin_31.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
1,0,1,1,3424,2.30,1.70,0.410720,0.589280
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
3,0,3,0,8144,1.55,2.30,0.592345,0.407655
4,0,4,0,6559,2.00,2.00,0.641717,0.358283


## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


In [6]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
4,0,4,0,6559,2.00,2.00,0.641717,0.358283
6,0,6,1,2470,1.55,1.10,0.329730,0.670270
8,0,8,1,4669,1.55,1.70,0.419431,0.580569
10,0,10,0,3301,1.10,1.10,0.605079,0.394921
12,0,12,1,3117,1.85,1.10,0.506602,0.493398
14,0,14,0,2754,1.25,1.55,0.596632,0.403368
16,0,16,1,3931,1.85,1.70,0.338392,0.661608
18,0,18,0,5674,1.70,1.40,0.410655,0.589345


## Hierarchical GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_Ff2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 31 subjects...


INFO (theano.gof.compilelock): Waiting for existing lock by process '4634' (I am process '4633')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.2.0-x86_64-i386-64bit-i386-3.5.4-64/lock_dir



You can find the C code in this temporary file: /var/folders/np/3gkzqfkd6vddg92v1ytvrkkc0000gn/T/theano_compilation_error_nlwt1i7_


Exception: ("Compilation failed (return status=1): In file included from /Users/pradyumna/.theano/compiledir_Darwin-18.2.0-x86_64-i386-64bit-i386-3.5.4-64/tmphy400ot8/mod.cpp:1:. In file included from /Users/pradyumna/anaconda/include/python3.5m/Python.h:25:. /Users/pradyumna/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. ", '[Elemwise{add,no_inplace}(TensorConstant{0.01}, TensorConstant{1e-06})]')

In [ ]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_full'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

In [ ]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_F2019_full'+ sufix +'.npy'))

In [ ]:
# Compute WAICs
np.save(str('results/waic/glam_FF2019_full'+ sufix +'.npy'), glam_full.waic)

In [ ]:
len(glam_full.waic)

In [ ]:
glam_full.waic

In [ ]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2019_full'+ sufix +'.npy'), glam_full.loo)

In [ ]:
glam_full.loo

In [ ]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

### 1. no-bias GLAM

In [ ]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM hierarchically...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_nobias_'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_FF2019_nobias'+sufix+'.npy'))
 

In [ ]:
   
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_nobias_individual_cv'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

In [ ]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

In [ ]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_FF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

In [ ]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv'))

glam_nobias.prediction.head()

## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
glam.plots_pretty.plot_fit(test_data, [glam_full.prediction]);

#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## Parameters for full hierarchical model

In [ ]:
params_participant = glam_full.estimates
params_participant

In [ ]:
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 